In [36]:
file_path = 'data.txt'

# Initialize lists to store vehicle and customer data
vehicle_data = []
customer_data = []

# Read data line by line
with open(file_path, 'r') as file:
    lines = file.readlines()
    
    # Parse vehicle data
    vehicle_info_start = lines.index('VEHICLE\n') + 2
    vehicle_info_end = lines.index('CUSTOMER\n')
    vehicle_line = lines[vehicle_info_start].strip().split()
    
    # Store vehicle data as a dictionary in a list
    vehicle_data.append({
        'NUMBER': int(vehicle_line[0]),
        'CAPACITY': int(vehicle_line[1])
    })
    
    # Parse customer data
    customer_info_start = vehicle_info_end + 2
    for line in lines[customer_info_start:]:
        if line.strip():  # Skip any empty lines
            parts = line.strip().split()
            customer_data.append({
                'CUST_NO': int(parts[0]),
                'XCOORD': int(parts[1]),
                'YCOORD': int(parts[2]),
                'DEMAND': int(parts[3]),
                'READY_TIME': int(parts[4]),
                'DUE_DATE': int(parts[5]),
                'SERVICE_TIME': int(parts[6])
            })

# Display the parsed data
print("Vehicle Data:")
print(vehicle_data)
print("\nCustomer Data:")
for customer in customer_data:  # Show first 5 customers for brevity
    print(customer)


Vehicle Data:
[{'NUMBER': 25, 'CAPACITY': 200}]

Customer Data:
{'CUST_NO': 0, 'XCOORD': 40, 'YCOORD': 50, 'DEMAND': 0, 'READY_TIME': 0, 'DUE_DATE': 1236, 'SERVICE_TIME': 0}
{'CUST_NO': 1, 'XCOORD': 45, 'YCOORD': 68, 'DEMAND': 10, 'READY_TIME': 912, 'DUE_DATE': 967, 'SERVICE_TIME': 90}
{'CUST_NO': 2, 'XCOORD': 45, 'YCOORD': 70, 'DEMAND': 30, 'READY_TIME': 825, 'DUE_DATE': 870, 'SERVICE_TIME': 90}
{'CUST_NO': 3, 'XCOORD': 42, 'YCOORD': 66, 'DEMAND': 10, 'READY_TIME': 65, 'DUE_DATE': 146, 'SERVICE_TIME': 90}
{'CUST_NO': 4, 'XCOORD': 42, 'YCOORD': 68, 'DEMAND': 10, 'READY_TIME': 727, 'DUE_DATE': 782, 'SERVICE_TIME': 90}
{'CUST_NO': 5, 'XCOORD': 42, 'YCOORD': 65, 'DEMAND': 10, 'READY_TIME': 15, 'DUE_DATE': 67, 'SERVICE_TIME': 90}
{'CUST_NO': 6, 'XCOORD': 40, 'YCOORD': 69, 'DEMAND': 20, 'READY_TIME': 621, 'DUE_DATE': 702, 'SERVICE_TIME': 90}
{'CUST_NO': 7, 'XCOORD': 40, 'YCOORD': 66, 'DEMAND': 20, 'READY_TIME': 170, 'DUE_DATE': 225, 'SERVICE_TIME': 90}
{'CUST_NO': 8, 'XCOORD': 38, 'YCOORD':

1. Data Structures
From your parsed data:

vehicle_data: Contains the number of vehicles and their capacities.
customer_data: Includes customer coordinates, demands, time windows, and service times.
We'll convert this into a format usable for the GAH:

Distance Matrix: Precompute the distances between all customers (including the depot).
Demand, Time Windows, and Service Times: Use them as constraints in the evaluation function.

2. Distance Matrix Calculation
Compute the distance matrix between all customers (and the depot).

In [37]:
import numpy as np
from math import sqrt

# Compute the distance matrix
def calculate_distance_matrix(customer_data):
    num_customers = len(customer_data)
    distance_matrix = np.zeros((num_customers + 1, num_customers + 1))  # +1 for the depot
    all_points = [(0, 0)] + [(cust['XCOORD'], cust['YCOORD']) for cust in customer_data]  # Depot at (0,0)
    
    for i in range(len(all_points)):
        for j in range(len(all_points)):
            distance_matrix[i][j] = sqrt((all_points[i][0] - all_points[j][0])**2 +
                                         (all_points[i][1] - all_points[j][1])**2)
    return distance_matrix

distance_matrix = calculate_distance_matrix(customer_data)


3. Fitness Function
Define the fitness function to evaluate solutions. This includes:

Total Distance: Sum of all route distances.
Penalty for Constraints: Add large penalties for capacity and time window violations.

In [38]:
def evaluate_solution(solution, distance_matrix, vehicle_capacity, customer_data):
    total_cost = 0
    penalty = 0
    is_feasible = True
    
    for route in solution:
        load = 0
        time = 0
        prev_customer = 0
        for customer in route:
            if customer == 0:
                continue
            cust_data = customer_data[customer - 1]
            load += cust_data['DEMAND']
            time = max(time + distance_matrix[prev_customer][customer], cust_data['READY_TIME'])
            if load > vehicle_capacity or time > cust_data['DUE_DATE']:
                is_feasible = False
            time += cust_data['SERVICE_TIME']
            prev_customer = customer
        total_cost += distance_matrix[prev_customer][0]
    
    if not is_feasible:
        penalty += 1e6  # Pénalité élevée pour solutions non faisables
    
    return total_cost + penalty


4. Genetic Algorithm Framework
Here's the main structure of the GAH, incorporating the data you parsed.

In [39]:
import random

# Parameters
population_size = 50
generations = 100
mutation_rate = 0.15
vehicle_capacity = vehicle_data[0]['CAPACITY']
num_vehicles = vehicle_data[0]['NUMBER']
num_customers = len(customer_data)

# Generate initial population
def create_initial_solution():
    """Générer une solution faisable en respectant les contraintes de capacité et de temps."""
    customers = list(range(1, num_customers + 1))
    random.shuffle(customers)
    solution = []

    for _ in range(num_vehicles):
        route = [0]  # Commence au dépôt
        load = 0
        time = 0
        remaining_customers = []
        
        for customer in customers:
            cust_data = customer_data[customer - 1]
            demand = cust_data['DEMAND']
            ready_time = cust_data['READY_TIME']
            due_date = cust_data['DUE_DATE']
            service_time = cust_data['SERVICE_TIME']
            travel_time = distance_matrix[route[-1]][customer]

            # Vérifie la faisabilité d'ajouter ce client
            arrival_time = max(time + travel_time, ready_time)
            if load + demand <= vehicle_capacity and arrival_time <= due_date:
                route.append(customer)
                load += demand
                time = arrival_time + service_time
            else:
                remaining_customers.append(customer)

        route.append(0)  # Retour au dépôt
        solution.append(route)
        customers = remaining_customers

    # Vérifie si tous les clients ont été servis
    if customers:
        raise ValueError("Impossible de générer une solution initiale faisable avec les véhicules disponibles.")
    
    return solution



# Genetic operators
def crossover(parent1, parent2):
    """Partially Matched Crossover (PMX) avec réparation."""
    child = []
    for route1, route2 in zip(parent1, parent2):
        size = len(route1)
        if size <= 3:  # Cas de routes courtes
            child.append(route1[:])  # Copie directe
        else:
            start, end = sorted(random.sample(range(1, size - 1), 2))
            child_part = route1[start:end]
            child_route = [c for c in route2 if c not in child_part]
            child.append([0] + child_route[:start] + child_part + child_route[start:] + [0])
    return repair_solution(child)



def mutate(solution):
    """Mutation par échange avec réparation."""
    for route in solution:
        if random.random() < mutation_rate:
            i, j = random.sample(range(1, len(route) - 1), 2)
            route[i], route[j] = route[j], route[i]
    return repair_solution(solution)


def local_search(solution):
    """2-opt optimization for routes."""
    for route in solution:
        best_cost = evaluate_solution([route], distance_matrix, vehicle_capacity, customer_data)
        for i in range(1, len(route) - 2):
            for j in range(i + 1, len(route) - 1):
                new_route = route[:i] + route[i:j + 1][::-1] + route[j + 1:]
                new_cost = evaluate_solution([new_route], distance_matrix, vehicle_capacity, customer_data)
                if new_cost < best_cost:
                    route = new_route
                    best_cost = new_cost
    return solution


def repair_solution(solution):
    """Réparer une solution non faisable pour respecter les contraintes."""
    repaired_solution = []
    unassigned_customers = []
    
    for route in solution:
        load = 0
        time = 0
        new_route = [0]  # Commence au dépôt
        
        for customer in route[1:-1]:  # Ignore le dépôt (0)
            cust_data = customer_data[customer - 1]
            demand = cust_data['DEMAND']
            ready_time = cust_data['READY_TIME']
            due_date = cust_data['DUE_DATE']
            service_time = cust_data['SERVICE_TIME']
            travel_time = distance_matrix[new_route[-1]][customer]

            arrival_time = max(time + travel_time, ready_time)
            if load + demand <= vehicle_capacity and arrival_time <= due_date:
                new_route.append(customer)
                load += demand
                time = arrival_time + service_time
            else:
                unassigned_customers.append(customer)
        
        new_route.append(0)  # Retour au dépôt
        repaired_solution.append(new_route)
    
    # Assigne les clients non affectés à de nouveaux véhicules
    while unassigned_customers:
        route = [0]
        load = 0
        time = 0
        remaining_customers = []
        
        for customer in unassigned_customers:
            cust_data = customer_data[customer - 1]
            demand = cust_data['DEMAND']
            ready_time = cust_data['READY_TIME']
            due_date = cust_data['DUE_DATE']
            service_time = cust_data['SERVICE_TIME']
            travel_time = distance_matrix[route[-1]][customer]

            arrival_time = max(time + travel_time, ready_time)
            if load + demand <= vehicle_capacity and arrival_time <= due_date:
                route.append(customer)
                load += demand
                time = arrival_time + service_time
            else:
                remaining_customers.append(customer)
        
        route.append(0)  # Retour au dépôt
        repaired_solution.append(route)
        unassigned_customers = remaining_customers
    
    return repaired_solution


# Main GAH loop
def genetic_algorithm():
    population = [create_initial_solution() for _ in range(population_size)]
    
    for generation in range(generations):
        new_population = []
        for _ in range(population_size // 2):
            parent1, parent2 = random.sample(population, 2)
            child1, child2 = crossover(parent1, parent2), crossover(parent2, parent1)
            child1, child2 = mutate(child1), mutate(child2)
            child1, child2 = local_search(child1), local_search(child2)
            new_population.extend([child1, child2])
        population = sorted(population + new_population, key=lambda sol: evaluate_solution(sol, distance_matrix, vehicle_capacity, customer_data))[:population_size]
    
    best_solution = min(population, key=lambda sol: evaluate_solution(sol, distance_matrix, vehicle_capacity, customer_data))
    best_cost = evaluate_solution(best_solution, distance_matrix, vehicle_capacity, customer_data)
    return best_solution, best_cost

# Run GAH
best_solution, best_cost = genetic_algorithm()
print("Best Solution:", best_solution)
print("Best Cost:", best_cost)


ValueError: Impossible de générer une solution initiale faisable avec les véhicules disponibles.

Next Steps
Visualization: Add a function to plot the routes.
Dynamic Feasibility Checking: Improve handling of capacity and time window constraints dynamically during solution generation.
Performance Tuning: Adjust parameters like population size, mutation rate, and local search strategy for optimal results.